In [1]:
!pip install openrouteservice
!pip install ortools

You should consider upgrading via the 'c:\users\d6002091\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\d6002091\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
from __future__ import print_function
from IPython.core.display import clear_output
from os import path
from random import randint
from warnings import warn
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import requests
import numpy as np
import seaborn as sns
import string
import warnings
import folium
import time
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import trange, tqdm, tnrange
from folium.plugins import HeatMap
import openrouteservice
from openrouteservice.directions import directions
import geopy.distance
from math import sin, cos, sqrt, atan2, radians
from ortools.constraint_solver import routing_enums_pb2, pywrapcp

warnings.filterwarnings('ignore')

# **Data Load and preperation**

In [3]:
#Lets load the data
Standorte = pd.read_csv('https://www.get-in-it.de/imgs/it/codingCompetition/msg/msg_standorte_deutschland.csv', engine='python', encoding='utf-8', index_col=0)
Standorte.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 1 to 21
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   msg Standort  21 non-null     object 
 1   Straße        21 non-null     object 
 2   Hausnummer    21 non-null     object 
 3   PLZ           21 non-null     int64  
 4   Ort           21 non-null     object 
 5   Breitengrad   21 non-null     float64
 6   Längengrad    21 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.3+ KB


In [5]:
#Some data wrangling
Standorte['Location'] = Standorte['msg Standort']+", "+Standorte['Straße']+" "+Standorte['Hausnummer']+", "+Standorte['PLZ'].map(str)+", "+Standorte['Ort']
Standorte["color"] = "blue"
Standorte.loc[1,'color'] = "green"
Standorte.rename( columns={'Breitengrad':'latitude'}, inplace=True )
Standorte.rename( columns={'Längengrad':'longitude'}, inplace=True )

# **Look on the maps**

In [6]:
#First display of the data
base_map = folium.Map(location=[51.1633987,10.4468737], zoom_start=6.2)

locations = Standorte[['latitude', 'longitude']]
locationlist = locations.values.tolist()
namelist = Standorte.Location.values.tolist()
colorlist = Standorte.color.values.tolist()

len(namelist)
len(locationlist)

#add points
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=namelist[point], icon=folium.Icon(color=colorlist[point])).add_to(base_map)

# add tile layers to the map
tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']

for tile in tiles:
    folium.TileLayer(tile).add_to(base_map)
    
folium.LayerControl().add_to(base_map)
    
base_map

In [7]:
#How do the points look on the map including routes?
base_map = folium.Map(location=[51.1633987,10.4468737], zoom_start=6.2)

#add points
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=namelist[point], icon=folium.Icon(color=colorlist[point])).add_to(base_map)
    
#add lines
pointlist = [tuple(x) for x in locations.to_numpy()]
    
folium.PolyLine(pointlist, color="red", weight=2.5, opacity=1).add_to(base_map)

# add tile layers to the map
tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']

for tile in tiles:
    folium.TileLayer(tile).add_to(base_map)
    
folium.LayerControl().add_to(base_map)
    
base_map

# **How long is the route if we are flying the route as displayed in the list?**

In [8]:
Standorte_temp = Standorte.copy()
Standorte_temp.index -= 1
Standorte['Key'] = Standorte.index
Standorte_temp['Key'] = Standorte_temp.index
Standorte_temp = Standorte.merge(Standorte_temp[['latitude','longitude','Key']], how='inner', on='Key')

R = 6373.0

distances = []
for point in range(0, len(Standorte_temp['latitude_x'])):
    lat1 = radians(Standorte_temp['latitude_x'][point])
    lon1 = radians(Standorte_temp['longitude_x'][point])
    lat2 = radians(Standorte_temp['latitude_y'][point])
    lon2 = radians(Standorte_temp['longitude_y'][point])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    distances.append(distance)
    
print (str((round(sum(distances))))+" KM")

Standorte_temp['Distance_fly'] = distances

6175 KM


# **How long is the route if we are driving the route as displayed in the list?**

In [9]:
#find route
coords = list(zip(Standorte['longitude'],Standorte['latitude']))[0:11]
client = openrouteservice.Client(key='') # Specify your personal API key
request = {'coordinates': coords,
           'profile': 'driving-car',
           'geometry': 'true',
           'format_out': 'geojson',
            'instructions': 'false'}
routes_1 = client.directions(**request)

coords = list(zip(Standorte['longitude'],Standorte['latitude']))[10:21]
client = openrouteservice.Client(key='') # Specify your personal API key
request = {'coordinates': coords,
           'profile': 'driving-car',
           'geometry': 'true',
           'format_out': 'geojson',
            'instructions': 'false'}
routes_2 = client.directions(**request)

In [12]:
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)

base_map = folium.Map(location=[51.1633987,10.4468737], zoom_start=6)

#add points
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=namelist[point], icon=folium.Icon(color=colorlist[point])).add_to(base_map)

#add route to map
folium.features.GeoJson(data=routes_1,
                        name='Test', style_function=style_function('#6666ff'),
                       overlay=True).add_to(base_map)

folium.features.GeoJson(data=routes_2,
                        name='Test', style_function=style_function('#6666ff'),
                       overlay=True).add_to(base_map)

# add tile layers to the map
tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']

for tile in tiles:
    folium.TileLayer(tile).add_to(base_map)
    
folium.LayerControl().add_to(base_map)
    
base_map

In [11]:
distance1, duration1 = routes_1['features'][0]['properties']['summary'].values()
distance2, duration2 = routes_2['features'][0]['properties']['summary'].values()

print(str(round((distance1+distance2)/1000))+" KM")
print(str(round((duration1+duration2)/60))+" mins")

7875 KM
4265 mins


## **Lets optimize this, we start with a cross table to display all possibilites routes. How long is the sum of all possible routes if we are flying?**

In [13]:
Standorte['key'] = 0
Standorte_cross = Standorte.copy()
Standorte_cross['key'] = 0

Standorte_cross = Standorte_cross.merge(Standorte, how='outer', on='key')
Standorte_cross = Standorte_cross[Standorte_cross['msg Standort_x'] != Standorte_cross['msg Standort_y']]
Standorte_cross = Standorte_cross.reset_index(drop=True)

distances = []
for point in range(0, len(Standorte_cross['latitude_x'])):
    lat1 = radians(Standorte_cross['latitude_x'][point])
    lon1 = radians(Standorte_cross['longitude_x'][point])
    lat2 = radians(Standorte_cross['latitude_y'][point])
    lon2 = radians(Standorte_cross['longitude_y'][point])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    distances.append(distance)
    
print (str((round(sum(distances))))+" KM")

Standorte_cross['Distance_fly'] = distances

140631 KM


## **And how long is the sum of all possible routes if we are driving?**

In [17]:
distance_final = []
duration_final = []

for point in range(0, len(Standorte_cross['latitude_x'])):
    routes= []
    distance = []
    duration = []
    
    coords = list(zip(Standorte_cross['longitude_x'],Standorte_cross['latitude_x']))[point:point+1]+(list(zip(Standorte_cross['longitude_y'],Standorte_cross['latitude_y']))[point:point+1])
    client = openrouteservice.Client(key='') # Specify your personal API key
    request = {'coordinates': coords,
               'profile': 'driving-car',
               'geometry': 'true',
               'format_out': 'geojson',
                'instructions': 'false'}
    routes = client.directions(**request)
    
    distance, duration = routes['features'][0]['properties']['summary'].values()
    
    distance = round((distance)/1000)
    duration = round((duration)/60)
    
    distance_final.append(distance)
    duration_final.append(duration)
    
Standorte_cross['Distance_car'] = distance_final
Standorte_cross['Duration_car'] = duration_final

print(str(sum(distance_final))+" KM")
print(str(sum(duration_final))+" mins")

179307 KM
97030 mins


In [14]:
#Please use this prepared list if routing above fails
Standorte_cross = pd.read_csv('https://github.com/arnimpdm/Get_in_IT_Travelling_Salesman_Problem/raw/master/Standorte_cross.CSV', engine='python', encoding='utf-8', index_col=0)
Standorte_cross.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 419
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   msg Standort_x  420 non-null    object 
 1   Straße_x        420 non-null    object 
 2   Hausnummer_x    420 non-null    object 
 3   PLZ_x           420 non-null    int64  
 4   Ort_x           420 non-null    object 
 5   latitude_x      420 non-null    float64
 6   longitude_x     420 non-null    float64
 7   Location_x      420 non-null    object 
 8   color_x         420 non-null    object 
 9   Key_x           420 non-null    int64  
 10  key             420 non-null    int64  
 11  msg Standort_y  420 non-null    object 
 12  Straße_y        420 non-null    object 
 13  Hausnummer_y    420 non-null    object 
 14  PLZ_y           420 non-null    int64  
 15  Ort_y           420 non-null    object 
 16  latitude_y      420 non-null    float64
 17  longitude_y     420 non-null    flo

# **Lets Pivot the data and then put it into the TSP solver**

# In order to have a real life example we will be using the street distances going forward.

In [15]:
Standorte_pivot = Standorte_cross.pivot(index='msg Standort_y',columns='msg Standort_x', values='Distance_car')
Standorte_pivot = Standorte_pivot.reindex(['Ismaning/München (Hauptsitz)','Berlin','Braunschweig','Bretten','Chemnitz','Düsseldorf','Essen','Frankfurt','Görlitz','Hamburg','Hannover','Ingolstadt','Köln/Hürth','Lingen (Ems)','Münster','Nürnberg','Passau','Schortens/Wilhelmshaven','St. Georgen','Stuttgart','Walldorf'], axis=0)
Standorte_pivot = Standorte_pivot.reindex(['Ismaning/München (Hauptsitz)','Berlin','Braunschweig','Bretten','Chemnitz','Düsseldorf','Essen','Frankfurt','Görlitz','Hamburg','Hannover','Ingolstadt','Köln/Hürth','Lingen (Ems)','Münster','Nürnberg','Passau','Schortens/Wilhelmshaven','St. Georgen','Stuttgart','Walldorf'], axis=1)

In [16]:
#TSP solver

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = Standorte_pivot.values
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

def print_solution(manager, routing, assignment):
    """Prints assignment on console."""
    print('Total distance: {} kilometers'.format(assignment.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Index:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    
    
def return_indexes(routing, assignment):
    index = routing.Start(0)
    indexes = []
    while not routing.IsEnd(index):
        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        indexes = np.append(indexes, index)
    return indexes
    
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print_solution(manager, routing, assignment)
        indexes = return_indexes(routing, assignment)
    return indexes

if __name__ == '__main__':
    Index = main()

#Source: https://developers.google.com/optimization/routing/tsp

Total distance: 3197 kilometers
Index:
 0 -> 16 -> 4 -> 8 -> 1 -> 2 -> 10 -> 9 -> 17 -> 13 -> 14 -> 6 -> 5 -> 12 -> 7 -> 20 -> 3 -> 19 -> 18 -> 15 -> 11 -> 0



# **We will use the generated index for our solution list**

In [17]:
#Index[-1:] = 0
Index = np.insert(Index, 0, 0)
sorterIndex = dict(zip(Index,range(len(Index))))
sorterIndex

{0.0: 0,
 16.0: 1,
 4.0: 2,
 8.0: 3,
 1.0: 4,
 2.0: 5,
 10.0: 6,
 9.0: 7,
 17.0: 8,
 13.0: 9,
 14.0: 10,
 6.0: 11,
 5.0: 12,
 12.0: 13,
 7.0: 14,
 20.0: 15,
 3.0: 16,
 19.0: 17,
 18.0: 18,
 15.0: 19,
 11.0: 20,
 21.0: 21}

# **And we will map the final route....**

In [18]:
Standorte_solution = Standorte.copy()
Standorte_solution['Index'] = Standorte_solution['Key']-1
Standorte_solution['Index_sorted'] = Standorte_solution['Index'].map(sorterIndex)
Standorte_solution = Standorte_solution.sort_values(by='Index_sorted')
Standorte_solution_ = pd.DataFrame(Standorte_solution[0:1].values, columns=Standorte_solution.columns)
Standorte_solution = Standorte_solution.append(Standorte_solution_)
Standorte_solution = Standorte_solution.reset_index(drop=True)
Standorte_solution

,msg Standort,Straße,Hausnummer,PLZ,Ort,latitude,longitude,Location,color,Key,key,Index,Index_sorted
0,Ismaning/München (Hauptsitz),Robert-Bürkle-Straße,1,85737,Ismaning,48.229,11.6862,"Ismaning/München (Hauptsitz), Robert-Bürkle-St...",green,1,0,0,0
1,Passau,Dr. Hans-Kapfinger-Straße,30,94032,Passau,48.572,13.4533,"Passau, Dr. Hans-Kapfinger-Straße 30, 94032, P...",blue,17,0,16,1
2,Chemnitz,Zwickauer Straße,16a,9122,Chemnitz,50.8294,12.9147,"Chemnitz, Zwickauer Straße 16a, 9122, Chemnitz",blue,5,0,4,2
3,Görlitz,Melanchthonstraße,19,2826,Görlitz,51.1455,14.97,"Görlitz, Melanchthonstraße 19, 2826, Görlitz",blue,9,0,8,3
4,Berlin,Wittestraße,30,13509,Berlin,52.5809,13.2939,"Berlin, Wittestraße 30, 13509, Berlin",blue,2,0,1,4
5,Braunschweig,Mittelweg,7,38106,Braunschweig,52.2787,10.5248,"Braunschweig, Mittelweg 7, 38106, Braunschweig",blue,3,0,2,5
6,Hannover,Hildesheimerstraße,265-267,30519,Hannover,52.338,9.76971,"Hannover, Hildesheimerstraße 265-267, 30519, H...",blue,11,0,10,6
7,Hamburg,Dammtorwall,7a,20354,Hamburg,53.5576,9.98606,"Hamburg, Dammtorwall 7a, 20354, Hamburg",blue,10,0,9,7
8,Schortens/Wilhelmshaven,Beethovenstraße,46,26419,Schortens,53.5378,7.93681,"Schortens/Wilhelmshaven, Beethovenstraße 46, 2...",blue,18,0,17,8
9,Lingen (Ems),Kaiserstraße,10b,49809,Lingen,52.5192,7.32219,"Lingen (Ems), Kaiserstraße 10b, 49809, Lingen",blue,14,0,13,9


In [19]:
#find route
coords = list(zip(Standorte_solution['longitude'],Standorte_solution['latitude']))[0:11]
client = openrouteservice.Client(key='') # Specify your personal API key
request = {'coordinates': coords,
           'profile': 'driving-car',
           'geometry': 'true',
           'format_out': 'geojson',
            'instructions': 'false'}
routes_1 = client.directions(**request)

coords = list(zip(Standorte_solution['longitude'],Standorte_solution['latitude']))[10:22]
client = openrouteservice.Client(key='') # Specify your personal API key
request = {'coordinates': coords,
           'profile': 'driving-car',
           'geometry': 'true',
           'format_out': 'geojson',
            'instructions': 'false'}
routes_2 = client.directions(**request)

In [20]:
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)

base_map = folium.Map(location=[51.1633987,10.4468737], zoom_start=6)

#add points
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=namelist[point], icon=folium.Icon(color=colorlist[point])).add_to(base_map)

#add route to map
folium.features.GeoJson(data=routes_1,
                        name='Test', style_function=style_function('#6666ff'),
                       overlay=True).add_to(base_map)

folium.features.GeoJson(data=routes_2,
                        name='Test', style_function=style_function('#6666ff'),
                       overlay=True).add_to(base_map)

# add tile layers to the map
tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']

for tile in tiles:
    folium.TileLayer(tile).add_to(base_map)
    
folium.LayerControl().add_to(base_map)
    
base_map

# **And that is it, attached you find the final route and length.**

In [21]:
print(Standorte_solution.Location)

0     Ismaning/München (Hauptsitz), Robert-Bürkle-St...
1     Passau, Dr. Hans-Kapfinger-Straße 30, 94032, P...
2        Chemnitz, Zwickauer Straße 16a, 9122, Chemnitz
3          Görlitz, Melanchthonstraße 19, 2826, Görlitz
4                 Berlin, Wittestraße 30, 13509, Berlin
5        Braunschweig, Mittelweg 7, 38106, Braunschweig
6     Hannover, Hildesheimerstraße 265-267, 30519, H...
7               Hamburg, Dammtorwall 7a, 20354, Hamburg
8     Schortens/Wilhelmshaven, Beethovenstraße 46, 2...
9         Lingen (Ems), Kaiserstraße 10b, 49809, Lingen
10              Münster, Schulstraße 22, 48149, Münster
11              Essen, Am Thyssenhaus 1.3, 45128, Essen
12    Düsseldorf, Gladbecker Straße 3, 40472, Düssel...
13       Köln/Hürth, Max-Planck-Straße 40, 50354, Hürth
14    Frankfurt, Mergenthalerallee 73-75, 65760, Esc...
15          Walldorf, Altrottstraße 31, 69190, Walldorf
16              Bretten, Edisonstraße 2, 75015, Bretten
17    Stuttgart, Humboldtstraße 35, 70771, Leinf

In [22]:
Standorte_solution.to_csv('Standorte_solution.csv')

In [23]:
distance1, duration1 = routes_1['features'][0]['properties']['summary'].values()
distance2, duration2 = routes_1['features'][0]['properties']['summary'].values()

print(str(round((distance1+distance2)/1000))+" KM")
print(str(round((duration1+duration2)/60))+" mins")

4034 KM
2401 mins
